In [65]:
# -*- coding: utf-8 -*- 
from __future__ import unicode_literals

import numpy as np
import pandas as pd
import json
import itertools as itt
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns



import geopandas as gpd
import shapely
import fiona
import CoordinatesConverter as convert_xy  #the .py file for bd09 conversioin

%matplotlib inline
#plt.style.use('ggplot')
plt.rcParams['figure.figsize']=[18,8]
plt.rcParams['axes.titlesize'] = 'medium'
plt.rcParams['font.sans-serif'] = 'SimHei'
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['legend.frameon'] = False


WGS84 = 'EPSG:4326'
#UTM_bj = ccrs.UTM(50)
UTM_bj = 'EPSG:32650'
UTM_nj = 'EPSG:32650'


## set the projection (planar) crs of this area
CRS_planar = UTM_nj

# loading data

## overall boundary
the maximum extent for  blocks generation

In [2]:
boundary = gpd.read_file(r'D:\Research\Some Basic Data\Nanjing\cityborder_2035').to_crs(CRS_planar)
boundary = boundary.geometry.explode(index_parts=True).reset_index(drop=True).reset_index()
boundary['geometry'] = [shapely.geometry.Polygon(i) for i in boundary['geometry']]
boundary['geometry'] = boundary.buffer(0)

## blocks

In [ ]:
blocks = gpd.read_file('../data/blocks').to_crs(CRS_planar)

# voronoi diagram of block polygons (not used)
- voronoi diagram of polygons or lines can be approximated with voronoi of points on polygon boundaries   
but 
- how to deal with crossings?

In [3]:
def voronoi_diagram_polygons(data,densify_max_span=5, envelope=None,tolerance=0):
    # ============================densify the polygon boundary vertices========================
    #### retrieve the polygons exterior as well as interior
    geom = data.boundary.explode(index_parts=False)    

    #### explode into line segments
    def explode_line(temp):
        t = np.array(temp.coords)
        return [shapely.geometry.LineString([i,j]) for i,j in zip(t[:-1],t[1:])]
    geom = [j for i in geom for j in explode_line(i.simplify(tolerance=0.5))]
    geom = gpd.GeoSeries(geom,crs=data.crs).reset_index(name='geometry')

    #### divide long line segments
    geom['length'] = geom.length
    geom['divide'] = (geom['length']/densify_max_span).apply(np.ceil).astype(int)
    geom['geometry'] = [shapely.geometry.LineString([line.interpolate(p, normalized=True) for p in np.linspace(0,1,divide+1)]) \
                        for line,divide in zip(geom['geometry'],geom['divide'])]

    #### merge into whole
    geom = shapely.ops.linemerge(list(geom['geometry']))
    
    
    
    # ====================direct voronoi of the densified polygon boundaries=====================
    temp = shapely.ops.voronoi_diagram(geom, tolerance=tolerance)
    temp = gpd.GeoDataFrame({'geometry':list(temp.geoms)},crs=data.crs)
    temp['geometry'] = temp['geometry'].buffer(0) ### the returned geometry may be invalid! so first buffer(0) to make them valid
    
    
    
    # =======================join voronoi result with original polygon and dissolve============= 
    s = temp.sjoin(data,how='inner').dissolve('index_right')
    
    
    # ====================================clip by envelope======================================
    if envelope is not None:
        s = s.clip(envelope)
    
    s = s['geometry'].reset_index(drop=True).reset_index()
    return s

In [285]:
s = blocks.sjoin(boundary)

blocks1 = blocks[s['index']==0].copy()
blocks2 = blocks[s['index']==1].copy()



blocks1_vornoi = voronoi_diagram_polygons(blocks1,envelope=boundary['geometry'][0],tolerance=0.1)
blocks2_vornoi = voronoi_diagram_polygons(blocks2,envelope=boundary['geometry'][1],tolerance=0.1)

blocks_voronoi = pd.concat([blocks1_vornoi,blocks2_vornoi],ignore_index=True)
#blocks_voronoi.to_file('../data/blocks_voronoi')

C:\ProgramData\Anaconda3\envs\pycaret\lib\site-packages\geopandas\geodataframe.py:1327: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\ProgramData\Anaconda3\envs\pycaret\lib\site-packages\geopandas\geodataframe.py:1327: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)


# skeletonize, e.g. medial axis transform (MAT), chordal axis

[This package](https://github.com/NRCan/geo_sim_processing/blob/7894e6913d6ac51781e4656baf5bca17925efaec/chordal_axis_algorithm.py#L504) seems relevant. It employs chordal axis, and ___deals with crossing correction___. But it is in QGIS.  
The result of this package seems __reasonable__.
## generate the separation polygon for processing in QGIS
Note:
- fist check the geometry of generated blocks, or tessellation in QGIS would report error
- better to buffer out the overall boundary, making generated chordal axis longer than the actual overall boundary, easy for cut
    - do not buffer out too much (about 500m), or chordal axis calculation in QGIS would report error
- should densify polygon boundary to produce finer result
- better to export separation polygon separately into QGIS, e.g. 
    - first QGIS processes the main separation polygon (only one big polygon), then other separtion polygons (perhaps several polygons)
    - finally combine the result

In [4]:
def densify_polygon_boundary(polygon, densify_max_span=5):
    densified = shapely.geometry.Polygon(shell = densify_line(polygon.exterior, densify_max_span=densify_max_span),
                                         holes = [densify_line(i, densify_max_span=densify_max_span) for i in polygon.interiors])
    return densified

def densify_line(line, densify_max_span=5):    

    line = line.simplify(tolerance=0.5)  ##simplify to remove vertices that are extremely close
    t = np.array(line.coords)

    vertices = []
    for i,j in zip(t[:-1],t[1:]):
        segment = shapely.geometry.LineString([i,j])
        divide = int(np.ceil(segment.length/densify_max_span))
        vertices += [segment.interpolate(p, normalized=True) for p in np.linspace(0,1,divide,endpoint=False)] ##middle segments including start without end
    vertices.append(shapely.geometry.Point(t[-1]))  ##add the last point

    densified = shapely.geometry.LineString(vertices)
    return densified

In [5]:
blocks['geometry'] = [densify_polygon_boundary(i) for i in blocks['geometry']]

In [6]:
boundary1 = boundary[:1]
boundary2 = boundary[1:]

s = blocks.sjoin(boundary1)
blocks1 = blocks[blocks.index.isin(s.index)].copy()

s = blocks.sjoin(boundary2)
blocks2 = blocks[blocks.index.isin(s.index)].copy()

In [37]:
bg1 = boundary1.buffer(500,join_style=2)
bg2 = boundary2.buffer(500,join_style=2) 

In [42]:
sep = densify_polygon_boundary(bg1.unary_union).difference(blocks1.unary_union)
sep = gpd.GeoDataFrame({'geometry':list(sep.geoms)},crs=blocks.crs)
sep['geometry'] = sep['geometry'].buffer(0)
sep['area'] = sep.area
sep = sep.sort_values('area',ascending=False)

In [44]:
sep.iloc[1:].to_file('../data/sep2')

C:\ProgramData\Anaconda3\envs\pycaret\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


## process QGIS result
- polygonize the chordal axis to form polygons
- cut polygons with boundary, as boundary is previously buffered out to elongate chordal axis
- spatial join the polygons with blocks to get the polygons that represent a real block (not some sliver polygons generated from boundary cut), to ensure that:
    - every block gets __one and only one__ polygon

In [57]:
upper_axis1 = gpd.read_file('../data/chordal axis/axis2.shp')
upper_axis2 = gpd.read_file('../data/chordal axis/axis3.shp')
upper_axis = pd.concat([upper_axis1,upper_axis2],ignore_index=True)

upper = shapely.ops.polygonize(upper_axis.unary_union)
upper = gpd.GeoDataFrame({'geometry':list(upper.geoms)},crs=upper_axis.crs)
upper = upper.overlay(boundary2)

s = upper.sjoin(blocks2,how='left')
upper = s[['blockID','geometry']].copy()

In [49]:
lower_axis1 = gpd.read_file('../data/chordal axis/lower_axis1.shp')
lower_axis2 = gpd.read_file('../data/chordal axis/lower_axis2.shp')
lower_axis = pd.concat([lower_axis1,lower_axis2],ignore_index=True)

lower = shapely.ops.polygonize(lower_axis.unary_union)
lower = gpd.GeoDataFrame({'geometry':list(lower)},crs=lower_axis.crs)
lower = lower.overlay(boundary1)

s = lower.sjoin(blocks1,how='left')
lower = s[['blockID','geometry']].copy()

In [101]:
adjoining_blocks = pd.concat([upper,lower],ignore_index=True).sort_values('blockID')
adjoining_blocks.to_file('../data/adjoining_blocks')

C:\ProgramData\Anaconda3\envs\pycaret\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
